# 2025 March Madness Kaggle Competition

- Goal of the Competition

- Submission Strategy

## 1. Import Python packages

In [65]:
import numpy as np
import pandas as pd

## 2. Explore the Data

In [66]:
# Regular Season Results
regular_season_results = pd.read_csv('competition_data/MRegularSeasonCompactResults.csv')
regular_season_results['margin'] = regular_season_results['WScore'] - regular_season_results['LScore']

regular_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,margin
0,1985,20,1228,81,1328,64,N,0,17
1,1985,25,1106,77,1354,70,H,0,7
2,1985,25,1112,63,1223,56,H,0,7
3,1985,25,1165,70,1432,54,H,0,16
4,1985,25,1192,86,1447,74,H,0,12
...,...,...,...,...,...,...,...,...,...
191791,2025,106,1461,69,1102,62,H,0,7
191792,2025,106,1462,76,1139,63,H,0,13
191793,2025,106,1466,80,1480,62,H,0,18
191794,2025,106,1468,94,1122,68,H,0,26


## 3. Data Preprocessing

### 3-1. Margin of Each Game by Team

In [67]:
W_results = regular_season_results[['Season', 'DayNum', 'WTeamID', 'WScore', 'WLoc', 'margin']] \
    .rename(columns={'WTeamID': 'TeamID', 'WScore': 'Score', 'WLoc': 'Loc'}, inplace=False)

L_results = regular_season_results[['Season', 'DayNum', 'LTeamID', 'LScore', 'WLoc', 'margin']] \
    .rename(columns={'LTeamID': 'TeamID', 'LScore': 'Score', 'WLoc': 'Loc'}, inplace=False)

# change the sign of the margin
L_results['margin'] = - L_results['margin']

regular_season_results_flatten = pd.concat([W_results, L_results], axis=0)
regular_season_results_flatten

,Season,DayNum,TeamID,Score,Loc,margin
0,1985,20,1228,81,N,17
1,1985,25,1106,77,H,7
2,1985,25,1112,63,H,7
3,1985,25,1165,70,H,16
4,1985,25,1192,86,H,12
...,...,...,...,...,...,...
191791,2025,106,1102,62,H,-7
191792,2025,106,1139,63,H,-13
191793,2025,106,1480,62,H,-18
191794,2025,106,1122,68,H,-26


### 3-2. Margin of Each Team by Season

In [68]:
from sklearn.preprocessing import MinMaxScaler

regular_season_results_by_team = regular_season_results_flatten.groupby(by=['Season', 'TeamID'],
                                                                        as_index=False) \
                                                               .margin \
                                                               .mean() \
                                                               .sort_values(by='Season',
                                                                            ascending=False)
regular_season_results_by_team

,Season,TeamID,margin
13387,2025,1480,-11.444444
13137,2025,1222,17.769231
13139,2025,1224,-5.090909
13140,2025,1225,-4.041667
13141,2025,1226,-1.333333
...,...,...,...
183,1985,1337,5.407407
182,1985,1336,-8.250000
181,1985,1335,1.076923
180,1985,1334,-9.115385


### 3-3. Scaling

In [69]:
# Min-Max scaling
margin_min = regular_season_results_by_team.groupby('Season').margin.transform('min')
margin_max = regular_season_results_by_team.groupby('Season').margin.transform('max')

regular_season_results_by_team['margin_scale'] = (regular_season_results_by_team['margin'] - margin_min) / (margin_max - margin_min) 

regular_season_results_by_team

,Season,TeamID,margin,margin_scale
13387,2025,1480,-11.444444,0.399411
13137,2025,1222,17.769231,0.954676
13139,2025,1224,-5.090909,0.520173
13140,2025,1225,-4.041667,0.540116
13141,2025,1226,-1.333333,0.591593
...,...,...,...,...
183,1985,1337,5.407407,0.770956
182,1985,1336,-8.250000,0.466047
181,1985,1335,1.076923,0.674275
180,1985,1334,-9.115385,0.446726


## 4. Prediction

In [70]:
# Submission formatting
df_prediction = pd.merge(left=regular_season_results_by_team,
                         right=regular_season_results_by_team, 
                         how='left', on=['Season'])
df_prediction = df_prediction[df_prediction['TeamID_x'] < df_prediction['TeamID_y']]

# Labeling
df_prediction['ID'] = df_prediction['Season'].astype(str) \
                    + '_' + df_prediction['TeamID_x'].astype(str) \
                    + '_' + df_prediction['TeamID_y'].astype(str)

In [71]:
# Make a prediction based on the margin
df_prediction['Pred'] = (df_prediction['margin_scale_x']
                      / (df_prediction['margin_scale_x'] + df_prediction['margin_scale_y']))

df_prediction

,Season,TeamID_x,margin_x,margin_scale_x,TeamID_y,margin_y,margin_scale_y,ID,Pred
364,2025,1222,17.769231,0.954676,1480,-11.444444,0.399411,2025_1222_1480,0.705033
366,2025,1222,17.769231,0.954676,1224,-5.090909,0.520173,2025_1222_1224,0.647304
367,2025,1222,17.769231,0.954676,1225,-4.041667,0.540116,2025_1222_1225,0.638668
368,2025,1222,17.769231,0.954676,1226,-1.333333,0.591593,2025_1222_1226,0.617406
369,2025,1222,17.769231,0.954676,1227,0.500000,0.626439,2025_1222_1227,0.603799
...,...,...,...,...,...,...,...,...,...
4398336,1985,1102,-5.791667,0.520930,1338,2.576923,0.707764,1985_1102_1338,0.423971
4398337,1985,1102,-5.791667,0.520930,1337,5.407407,0.770956,1985_1102_1337,0.403232
4398338,1985,1102,-5.791667,0.520930,1336,-8.250000,0.466047,1985_1102_1336,0.527804
4398339,1985,1102,-5.791667,0.520930,1335,1.076923,0.674275,1985_1102_1335,0.435850


## 5. Validation

### 5-1. Historical Tournament Data

In [72]:
y_true = pd.read_csv("competition_data/MNCAATourneyCompactResults.csv")

y_true['Team1'] = y_true[['WTeamID', 'LTeamID']].min(axis=1)
y_true['Team2'] = y_true[['WTeamID', 'LTeamID']].max(axis=1)

y_true['ID'] = y_true.apply(lambda row: f"{row['Season']}_{row['Team1']}_{row['Team2']}", axis=1)
y_true['Result'] = y_true.apply(lambda row: 1 if row['WTeamID'] == row['Team1'] else 0, axis=1)

y_true.sort_values(by='ID')

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,ID,Result
16,1985,137,1104,50,1112,41,N,0,1104,1112,1985_1104_1112,1
52,1985,144,1301,61,1104,55,N,0,1104,1301,1985_1104_1301,0
40,1985,139,1104,63,1433,59,N,0,1104,1433,1985_1104_1433,1
0,1985,136,1116,63,1234,54,N,0,1116,1234,1985_1116_1234,1
39,1985,138,1385,68,1116,65,N,0,1116,1385,1985_1116_1385,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2483,2024,137,1361,69,1412,65,N,0,1361,1412,2024_1361_1412,1
2502,2024,139,1361,85,1463,57,N,0,1361,1463,2024_1361_1463,1
2468,2024,136,1397,83,1389,49,N,0,1389,1397,2024_1389_1397,0
2485,2024,137,1429,88,1395,72,N,0,1395,1429,2024_1395_1429,0


### 5-2. Calculate the Brier Score

In [73]:
df_tourney = pd.merge(left=y_true[['ID', 'Result']], right=df_prediction[['ID', 'Pred']], 
                  on='ID', how='inner')
df_tourney

,ID,Result,Pred
0,1985_1116_1234,1,0.452799
1,1985_1120_1345,1,0.499161
2,1985_1207_1250,1,0.644778
3,1985_1229_1425,1,0.516009
4,1985_1242_1325,1,0.507874
...,...,...,...
2513,2024_1181_1301,0,0.572122
2514,2024_1345_1397,1,0.511886
2515,2024_1104_1163,0,0.448280
2516,2024_1301_1345,0,0.422054


In [74]:
from sklearn.metrics import brier_score_loss

brier_score = brier_score_loss(df_tourney['Result'], df_tourney['Pred'])
print(f"Brier Score: {brier_score}")

Brier Score: 0.2316110934854095


## 6. Women's Tournament

In [87]:
## 1. Import Python packages
import numpy as np
import pandas as pd

## 2. Explore the Data
# Regular Season Results
regular_season_results_women = pd.read_csv('competition_data/WRegularSeasonCompactResults.csv')
regular_season_results_women['margin'] = regular_season_results_women['WScore'] - regular_season_results_women['LScore']

regular_season_results_women


## 3. Data Preprocessing
### 3-1. Margin of Each Game by Team
W_results_women = regular_season_results_women[['Season', 'DayNum', 'WTeamID', 'WScore', 'WLoc', 'margin']] \
    .rename(columns={'WTeamID': 'TeamID', 'WScore': 'Score', 'WLoc': 'Loc'}, inplace=False)

L_results_women = regular_season_results_women[['Season', 'DayNum', 'LTeamID', 'LScore', 'WLoc', 'margin']] \
    .rename(columns={'LTeamID': 'TeamID', 'LScore': 'Score', 'WLoc': 'Loc'}, inplace=False)

# change the sign of the margin
L_results_women['margin'] = - L_results_women['margin']

regular_season_results_women_flatten = pd.concat([W_results_women, L_results_women], axis=0)
regular_season_results_women_flatten
### 3-2. Margin of Each Team by Season
from sklearn.preprocessing import MinMaxScaler

regular_season_results_women_by_team = regular_season_results_women_flatten.groupby(by=['Season', 'TeamID'],
                                                                        as_index=False) \
                                                               .margin \
                                                               .mean() \
                                                               .sort_values(by='Season',
                                                                            ascending=False)
regular_season_results_women_by_team

### 3-3. Scaling
# Min-Max scaling
margin_min = regular_season_results_women_by_team.groupby('Season').margin.transform('min')
margin_max = regular_season_results_women_by_team.groupby('Season').margin.transform('max')

regular_season_results_women_by_team['margin_scale'] = (regular_season_results_women_by_team['margin'] - margin_min) / (margin_max - margin_min) 

regular_season_results_women_by_team

## 4. Prediction
# Submission formatting
df_prediction_women = pd.merge(left=regular_season_results_women_by_team,
                         right=regular_season_results_women_by_team, 
                         how='left', on=['Season'])
df_prediction_women = df_prediction_women[df_prediction_women['TeamID_x'] < df_prediction_women['TeamID_y']]

# Labeling
df_prediction_women['ID'] = df_prediction_women['Season'].astype(str) \
                    + '_' + df_prediction_women['TeamID_x'].astype(str) \
                    + '_' + df_prediction_women['TeamID_y'].astype(str)
# Make a prediction based on the margin
df_prediction_women['Pred'] = (df_prediction_women['margin_scale_x']
                      / (df_prediction_women['margin_scale_x'] + df_prediction_women['margin_scale_y']))

df_prediction_women
## 5. Validation
### 5-1. Historical Tournament Data
y_women_true = pd.read_csv("competition_data/WNCAATourneyCompactResults.csv")

y_women_true['Team1'] = y_women_true[['WTeamID', 'LTeamID']].min(axis=1)
y_women_true['Team2'] = y_women_true[['WTeamID', 'LTeamID']].max(axis=1)

y_women_true['ID'] = y_women_true.apply(lambda row: f"{row['Season']}_{row['Team1']}_{row['Team2']}", axis=1)
y_women_true['Result'] = y_women_true.apply(lambda row: 1 if row['WTeamID'] == row['Team1'] else 0, axis=1)

y_women_true.sort_values(by='ID')
### 5-2. Calculate the Brier Score
df_tourney_women = pd.merge(left=y_women_true[['ID', 'Result']], right=df_prediction_women[['ID', 'Pred']], 
                  on='ID', how='inner')
df_tourney_women

from sklearn.metrics import brier_score_loss

brier_score_women = brier_score_loss(df_tourney_women['Result'], df_tourney_women['Pred'])
print(f"Brier Score: {brier_score_women}")

Brier Score: 0.22328232287156646


## 7. Submission

In [89]:
submission_df = df_prediction[df_prediction.Season == 2025][['ID', 'Pred']]
submission_df_women = df_prediction_women[df_prediction_women.Season == 2025][['ID', 'Pred']]

submission_df = pd.concat([submission_df, submission_df_women], axis=0)
submission_df.sort_values(by='ID')

,ID,Pred
42706,2025_1101_1102,0.554610
42707,2025_1101_1103,0.406228
42708,2025_1101_1104,0.389226
42709,2025_1101_1105,0.550955
42710,2025_1101_1106,0.485716
...,...,...
89325,2025_3477_3479,0.450818
89052,2025_3477_3480,0.426532
89687,2025_3478_3479,0.276156
89414,2025_3478_3480,0.256878


In [ ]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


## References

- [Simple starter notebook for March Mania 2025](https://www.kaggle.com/code/paultimothymooney/simple-starter-notebook-for-march-mania-2025)
- [What is a Brier Score and How is it Calculated?](https://www.cultivatelabs.com/crowdsourced-forecasting-guide/what-is-a-brier-score-and-how-is-it-calculated)
